# Introduction to MPI

This section adapts [mpitutorial.com] materials using [IPython Parallel] and [mpi4py] to run MPI code in Jupyter notebooks.
We won't go into detail in using IPython Parallel, but cover the key bits for getting started.

[mpitutorial.com] materials are used under the MIT License.

[IPython Parallel]: https://ipyparallel.readthedocs.io

[mpitutorial.com]: https://mpitutorial.com

[mpi4py]: https://mpi4py.readthedocs.io/en/stable/

In [1]:
import ipyparallel as ipp

# create a cluster
cluster = ipp.Cluster(engines="mpi", n=2)
# start that cluster and connect to it
rc = cluster.start_and_connect_sync()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51engine/s]


What did that do?

```
mpiexec -n 2 python -m ipyparallel.engine --mpi
```

In [2]:
cluster.engine_set.args

['mpiexec',
 '-n',
 '2',
 '/Users/minrk/conda/envs/scan/bin/python',
 '-m',
 'ipyparallel.engine',
 '--mpi']

If we 'activate' the client,
it registers [magics with IPython](https://ipython.readthedocs.io/en/stable/interactive/magics.html), so we can use `%%px` to run cells on the _engines_
instead of in the local notebook.

In [3]:
rc.activate()

<DirectView all>

Now we have `%%px` available:

In [7]:
%%px
import os
pid = os.getpid()
pid

Out[0:5]: 37246

Out[1:5]: 37247

A cell passed to `%%px` is run on _all engines at once_.
This is the equivalent of `mpiexec myscript.py`, when running noninteractive MPI.

From now on, notebooks will start with a brief boilerplate to start and register the cluster,
so we can use `%%px`.

## Rank and size

Our very first MPI code, to test `%%px`.
We are going to get the "MPI World communicator".

The **rank** is the integer id of the current process,
while the **size** is the number of processes in the communicator.

In [6]:
%%px
# Find out rank, size
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size

print(f"I am rank {rank} / {size}")

[stdout:0] I am rank 0 / 2


[stdout:1] I am rank 1 / 2


In IPython Parallel, the state is _persistent_.
This means the `rank` and `size` variables can be used in subsequent cells:

In [9]:
%%px
print(f"Rank {rank} has PID {pid}")

[stdout:0] Rank 0 has PID 37246


[stdout:1] Rank 1 has PID 37247


To translate a notebook written with `%%px` to a script for `mpiexec`, you would concatenate all the `%%px` cells into a single `.py` file.


Now we can stop the cluster if we want.
It should get cleaned up automatically when the notebook exits, but it's good to be explicit.

In [11]:
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 37230, 'identifier': 'ipcontroller-1681284456-mwao-37042'}
Stopping engine(s): 1681284457
engine set stopped 1681284457: {'exit_code': 0, 'pid': 37244, 'identifier': 'ipengine-1681284456-mwao-1681284457-37042'}
